## Pengklasifikasi Masakan 2

Dalam pelajaran klasifikasi kedua ini, kita akan mengeksplorasi `lebih banyak cara` untuk mengklasifikasikan data kategorikal. Kita juga akan mempelajari dampak dari memilih satu pengklasifikasi dibandingkan yang lain.

### [**Kuis Pra-Pelajaran**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Prasyarat**

Kami mengasumsikan bahwa Anda telah menyelesaikan pelajaran sebelumnya karena kita akan melanjutkan beberapa konsep yang telah dipelajari sebelumnya.

Untuk pelajaran ini, kita memerlukan paket-paket berikut:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) adalah [koleksi paket R](https://www.tidyverse.org/packages) yang dirancang untuk membuat ilmu data lebih cepat, mudah, dan menyenangkan!

-   `tidymodels`: Kerangka kerja [tidymodels](https://www.tidymodels.org/) adalah [koleksi paket](https://www.tidymodels.org/packages/) untuk pemodelan dan pembelajaran mesin.

-   `themis`: Paket [themis](https://themis.tidymodels.org/) menyediakan Langkah-Langkah Resep Tambahan untuk Menangani Data yang Tidak Seimbang.

Anda dapat menginstalnya dengan perintah berikut:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Sebagai alternatif, skrip di bawah ini akan memeriksa apakah Anda sudah memiliki paket-paket yang diperlukan untuk menyelesaikan modul ini dan akan menginstalnya untuk Anda jika belum tersedia.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Peta klasifikasi**

Dalam [pelajaran sebelumnya](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), kita mencoba menjawab pertanyaan: bagaimana kita memilih di antara beberapa model? Sebagian besar, itu tergantung pada karakteristik data dan jenis masalah yang ingin kita selesaikan (misalnya klasifikasi atau regresi?).

Sebelumnya, kita telah mempelajari berbagai opsi yang tersedia saat mengklasifikasikan data menggunakan cheat sheet dari Microsoft. Kerangka kerja Machine Learning Python, Scikit-learn, menawarkan cheat sheet serupa tetapi lebih rinci yang dapat membantu mempersempit pilihan estimator Anda (istilah lain untuk classifier):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tip: [kunjungi peta ini secara online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) dan klik sepanjang jalur untuk membaca dokumentasi.  
>  
> [Situs referensi Tidymodels](https://www.tidymodels.org/find/parsnip/#models) juga menyediakan dokumentasi yang sangat baik tentang berbagai jenis model.

### **Rencana** 🗺️

Peta ini sangat membantu setelah Anda memiliki pemahaman yang jelas tentang data Anda, karena Anda dapat 'menelusuri' jalurnya untuk membuat keputusan:

-   Kami memiliki \>50 sampel

-   Kami ingin memprediksi sebuah kategori

-   Kami memiliki data berlabel

-   Kami memiliki kurang dari 100K sampel

-   ✨ Kami dapat memilih Linear SVC

-   Jika itu tidak berhasil, karena kami memiliki data numerik

    -   Kami dapat mencoba ✨ KNeighbors Classifier

        -   Jika itu juga tidak berhasil, coba ✨ SVC dan ✨ Ensemble Classifiers

Ini adalah jalur yang sangat membantu untuk diikuti. Sekarang, mari langsung mempraktikkannya menggunakan kerangka kerja [tidymodels](https://www.tidymodels.org/): kumpulan paket R yang konsisten dan fleksibel yang dikembangkan untuk mendorong praktik statistik yang baik 😊.

## 2. Membagi data dan menangani set data yang tidak seimbang.

Dari pelajaran sebelumnya, kita belajar bahwa ada sekumpulan bahan umum di berbagai masakan kita. Selain itu, terdapat distribusi yang cukup tidak merata dalam jumlah masakan.

Kita akan menangani ini dengan:

-   Menghapus bahan yang paling umum yang menciptakan kebingungan antara masakan yang berbeda, menggunakan `dplyr::select()`.

-   Menggunakan `recipe` yang memproses data agar siap untuk pemodelan dengan menerapkan algoritma `over-sampling`.

Kita sudah membahas hal di atas dalam pelajaran sebelumnya, jadi ini seharusnya mudah 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Mengatasi Data yang Tidak Seimbang

Data yang tidak seimbang sering kali berdampak negatif pada kinerja model. Banyak model bekerja paling baik ketika jumlah observasi seimbang, sehingga cenderung kesulitan dengan data yang tidak seimbang.

Ada dua cara utama untuk menangani data yang tidak seimbang:

-   menambahkan observasi ke kelas minoritas: `Over-sampling`, misalnya menggunakan algoritma SMOTE yang secara sintetis menghasilkan contoh baru dari kelas minoritas dengan menggunakan tetangga terdekat dari kasus-kasus tersebut.

-   menghapus observasi dari kelas mayoritas: `Under-sampling`

Dalam pelajaran sebelumnya, kami menunjukkan cara menangani data yang tidak seimbang menggunakan sebuah `recipe`. Recipe dapat dianggap sebagai cetak biru yang menjelaskan langkah-langkah apa saja yang harus diterapkan pada sebuah data set agar siap untuk analisis data. Dalam kasus kita, kita ingin memiliki distribusi yang seimbang dalam jumlah masakan untuk `training set` kita. Mari kita langsung mulai.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Sekarang kita siap untuk melatih model 👩‍💻👨‍💻!

## 3. Melampaui model regresi multinomial

Dalam pelajaran sebelumnya, kita telah mempelajari model regresi multinomial. Sekarang, mari kita jelajahi beberapa model yang lebih fleksibel untuk klasifikasi.

### Support Vector Machines

Dalam konteks klasifikasi, `Support Vector Machines` adalah teknik pembelajaran mesin yang berusaha menemukan *hyperplane* yang "paling baik" memisahkan kelas-kelas. Mari kita lihat contoh sederhana:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ tidak memisahkan kelas. H2~ memisahkan, tetapi hanya dengan margin kecil. H3~ memisahkan dengan margin maksimal.

#### Linear Support Vector Classifier

Support-Vector clustering (SVC) adalah bagian dari keluarga teknik ML Support-Vector machines. Dalam SVC, hyperplane dipilih untuk memisahkan `sebagian besar` pengamatan pelatihan dengan benar, tetapi `mungkin salah mengklasifikasikan` beberapa pengamatan. Dengan mengizinkan beberapa titik berada di sisi yang salah, SVM menjadi lebih tahan terhadap outlier sehingga lebih baik dalam melakukan generalisasi pada data baru. Parameter yang mengatur pelanggaran ini disebut `cost` yang memiliki nilai default 1 (lihat `help("svm_poly")`).

Mari kita buat SVC linear dengan mengatur `degree = 1` dalam model SVM polinomial.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Sekarang setelah kita menangkap langkah-langkah pra-pemrosesan dan spesifikasi model ke dalam *workflow*, kita dapat melanjutkan untuk melatih linear SVC dan mengevaluasi hasilnya sekaligus. Untuk metrik kinerja, mari kita buat satu set metrik yang akan mengevaluasi: `accuracy`, `sensitivity`, `Positive Predicted Value`, dan `F Measure`.

> `augment()` akan menambahkan kolom untuk prediksi ke data yang diberikan.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

Support vector machine (SVM) adalah pengembangan dari support vector classifier untuk mengakomodasi batas non-linear antara kelas. Pada dasarnya, SVM menggunakan *kernel trick* untuk memperbesar ruang fitur agar dapat menyesuaikan hubungan non-linear antara kelas. Salah satu fungsi kernel yang populer dan sangat fleksibel yang digunakan oleh SVM adalah *Radial basis function.* Mari kita lihat bagaimana performanya pada data kita.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Jauh lebih baik 🤩!

> ✅ Silakan lihat:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning dengan R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning dengan Aplikasi di R
>
> untuk bacaan lebih lanjut.

### Klasifikasi Tetangga Terdekat

*K*-nearest neighbor (KNN) adalah algoritma di mana setiap observasi diprediksi berdasarkan *kemiripannya* dengan observasi lainnya.

Mari kita terapkan algoritma ini pada data kita.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Tampaknya model ini tidak bekerja dengan baik. Mungkin mengubah argumen model (lihat `help("nearest_neighbor")`) akan meningkatkan kinerja model. Pastikan untuk mencobanya.

> ✅ Silakan lihat:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> untuk mempelajari lebih lanjut tentang classifier *K*-Nearest Neighbors.

### Classifier Ensemble

Algoritma ensemble bekerja dengan menggabungkan beberapa estimator dasar untuk menghasilkan model yang optimal, baik dengan cara:

`bagging`: menerapkan *fungsi rata-rata* pada kumpulan model dasar

`boosting`: membangun serangkaian model yang saling melengkapi untuk meningkatkan kinerja prediktif.

Mari kita mulai dengan mencoba model Random Forest, yang membangun kumpulan besar pohon keputusan kemudian menerapkan fungsi rata-rata untuk menghasilkan model keseluruhan yang lebih baik.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Kerja bagus 👏!

Mari kita juga bereksperimen dengan model Boosted Tree.

Boosted Tree mendefinisikan metode ensemble yang menciptakan serangkaian pohon keputusan secara berurutan, di mana setiap pohon bergantung pada hasil dari pohon sebelumnya dalam upaya untuk secara bertahap mengurangi kesalahan. Metode ini berfokus pada bobot item yang salah diklasifikasikan dan menyesuaikan model berikutnya untuk memperbaiki kesalahan tersebut.

Ada berbagai cara untuk menyesuaikan model ini (lihat `help("boost_tree")`). Dalam contoh ini, kita akan menyesuaikan Boosted Tree menggunakan mesin `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Silakan lihat:
>
> -   [Machine Learning untuk Ilmuwan Sosial](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning dengan R](https://bradleyboehmke.github.io/HOML/)
>
> -   [Pengantar Pembelajaran Statistik dengan Aplikasi di R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Membahas model AdaBoost yang merupakan alternatif bagus untuk xgboost.
>
> untuk mempelajari lebih lanjut tentang Ensemble classifiers.

## 4. Tambahan - membandingkan beberapa model

Kita telah membuat cukup banyak model dalam lab ini 🙌. Bisa menjadi melelahkan atau merepotkan untuk membuat banyak alur kerja dari berbagai set preprosesor dan/atau spesifikasi model, lalu menghitung metrik kinerja satu per satu.

Mari kita lihat apakah kita bisa mengatasi ini dengan membuat fungsi yang menyesuaikan daftar alur kerja pada set pelatihan, lalu mengembalikan metrik kinerja berdasarkan set pengujian. Kita akan menggunakan `map()` dan `map_dfr()` dari paket [purrr](https://purrr.tidyverse.org/) untuk menerapkan fungsi pada setiap elemen dalam daftar.

> Fungsi [`map()`](https://purrr.tidyverse.org/reference/map.html) memungkinkan Anda menggantikan banyak loop for dengan kode yang lebih ringkas dan lebih mudah dibaca. Tempat terbaik untuk mempelajari fungsi [`map()`](https://purrr.tidyverse.org/reference/map.html) adalah bab [iterasi](http://r4ds.had.co.nz/iteration.html) dalam R untuk ilmu data.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


Paket [**workflowset**](https://workflowsets.tidymodels.org/) memungkinkan pengguna untuk membuat dan dengan mudah melatih sejumlah besar model, tetapi sebagian besar dirancang untuk bekerja dengan teknik resampling seperti `cross-validation`, sebuah pendekatan yang belum kita bahas.

## **🚀Tantangan**

Setiap teknik ini memiliki sejumlah besar parameter yang dapat Anda sesuaikan, misalnya `cost` pada SVM, `neighbors` pada KNN, `mtry` (Prediktor yang Dipilih Secara Acak) pada Random Forest.

Teliti parameter default masing-masing dan pikirkan apa arti penyesuaian parameter ini terhadap kualitas model.

Untuk mengetahui lebih lanjut tentang model tertentu dan parameternya, gunakan: `help("model")` misalnya `help("rand_forest")`

> Dalam praktiknya, kita biasanya *mengestimasi* *nilai terbaik* untuk parameter ini dengan melatih banyak model pada `data set simulasi` dan mengukur seberapa baik kinerja semua model tersebut. Proses ini disebut **tuning**.

### [**Kuis setelah kuliah**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Tinjauan & Belajar Mandiri**

Ada banyak istilah teknis dalam pelajaran ini, jadi luangkan waktu sejenak untuk meninjau [daftar ini](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) yang berisi istilah-istilah penting!

#### TERIMA KASIH KEPADA:

[`Allison Horst`](https://twitter.com/allison_horst/) atas ilustrasi luar biasa yang membuat R lebih ramah dan menarik. Temukan lebih banyak ilustrasi di [galerinya](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) dan [Jen Looper](https://www.twitter.com/jenlooper) atas pembuatan versi Python asli dari modul ini ♥️

Selamat Belajar,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemah manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
